# pip Installtions

In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install transformers datasets evaluate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


# Imports

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from transformers.keras_callbacks import PushToHubCallback
from huggingface_hub import notebook_login

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import create_optimizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import backend as K
import sys
import os

2024-01-18 20:07:55.773880: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-18 20:07:55.809709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-18 20:07:55.994988: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-18 20:07:55.997714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 20:07:56.600041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

# GPU check

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2024-01-17 14:42:54.934699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-17 14:42:54.935073: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 17 14:42:57 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P3    27W /  55W |      6MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Hugging Face Login

In [ ]:
notebook_login()

# SET TOKENIZER

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
dataframe_completo = pd.read_csv("Datasets/out.csv")
ds_completo = Dataset.from_pandas(dataframe_completo)
ds_completo

Dataset({
    features: ['source', 'question', 'answer', 'context', 'language', 'label'],
    num_rows: 15164
})

In [10]:
y = dataframe_completo.loc[:,"label"]
X = dataframe_completo.loc[:,"question"]

In [11]:
X_train, X_to_split, y_train, y_to_split = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

In [12]:
X_validation, X_test, y_validation, y_test = train_test_split(X_to_split, y_to_split, test_size=0.5, random_state=7, stratify=y_to_split)

In [13]:
train_df = pd.concat([X_train, y_train], axis=1)
validation_df = pd.concat([X_validation, y_validation], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [14]:
train_df

,question,label
2404,Qual o dano nas algas está sendo causado pelo ...,1.0
12654,Em que setor os empregos começam a diminuir?,0.0
13630,Quando foi estabelecida a Direção da Bibliotec...,0.0
2560,Em quais as áreas é esperado que o ruído antro...,1.0
4582,Qual foi o tamanho da população da ilha de São...,1.0
...,...,...
7108,O Parque Nacional Monte Pascoal Ã© em qual bioma,0.0
10031,Que tipo de floresta Ã© a selva amazÃ´nica?,0.0
12231,Quantos colonos originais se estabeleceram em ...,0.0
379,Onde as pinças de Mithraculis foram coletadas?,1.0


In [15]:
train_ds = Dataset.from_pandas(train_df)
validation_ds = Dataset.from_pandas(validation_df)
test_ds = Dataset.from_pandas(test_df)

In [16]:
train_ds

Dataset({
    features: ['question', 'label', '__index_level_0__'],
    num_rows: 12131
})

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["question"], truncation=True)

In [18]:
tokenized_train_ds = train_ds.map(preprocess_function, batched=True)
tekenized_validation_ds = validation_ds.map(preprocess_function, batched=True)
tokenized_test_ds = test_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/12131 [00:00<?, ? examples/s]

Map:   0%|          | 0/1516 [00:00<?, ? examples/s]

Map:   0%|          | 0/1517 [00:00<?, ? examples/s]

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [20]:
# f1_metric = evaluate.load("f1")
# accuracy = evaluate.load("accuracy")

In [21]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

In [22]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [23]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# SYS Variables

In [ ]:
batch_size = int(sys.argv[1])
num_epochs = int(sys.argv[2])
learning_rate = float(sys.argv[3])
model_name = sys.argv[4]

# MODEL GENERAL

In [ ]:
# batch_size = 16
# num_epochs = 5
# learning_rate = 2e-5
# model_name = "bert-base-uncased"
output_name = model_name + "_" + learning_rate + "_" + num_epochs
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
push_to_hub_callback = PushToHubCallback(
    output_dir=output_name,
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_loss',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'min')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

# MODEL 1

In [24]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [25]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_primeiro",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model1.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_loss',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'min')
callbacks = [checkpoint, push_to_hub_callback]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Cloning https://huggingface.co/gustavokpc/IC_primeiro into local empty directory.


Download file tf_model.h5:   0%|          | 32.0k/418M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [26]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/5


2024-01-17 15:09:19.111516: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [12131]
	 [[{{node Placeholder/_0}}]]
2024-01-17 15:09:19.111879: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [12131]
	 [[{{node Placeholder/_0}}]]


 24/758 [..............................] - ETA: 12:29 - loss: 0.6652 - accuracy: 0.6068 - f1_m: 0.5543 - precision_m: 0.4148 - recall_m: 0.8763

KeyboardInterrupt: 

In [ ]:
val_acc = model.history.history['val_accuracy']
val_f1_m = model.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 2

In [ ]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model2 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set2 = model2.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set2 = model2.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model2.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set2)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_segundo",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model2.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model2.fit(x=tf_train_set2, validation_data=tf_validation_set2, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model2.history.history['val_accuracy']
val_f1_m = model2.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 3

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=3e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model3 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set3 = model3.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set3 = model3.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model3.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set3)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_terceiro",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model3.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model3.fit(x=tf_train_set3, validation_data=tf_validation_set3, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model3.history.history['val_accuracy']
val_f1_m = model3.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 4

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model4 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set4 = model4.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set4 = model4.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model4.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m]) # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set4)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_quarto",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model4.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model4.fit(x=tf_train_set4, validation_data=tf_validation_set4, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model4.history.history['val_accuracy']
val_f1_m = model4.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 5

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model5 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set5 = model5.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set5 = model5.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model5.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set5)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_quinto",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model5.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model5.fit(x=tf_train_set5, validation_data=tf_validation_set5, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model5.history.history['val_accuracy']
val_f1_m = model5.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 6

In [ ]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
with tf.device('/CPU:0'):
    model6 = TFAutoModelForSequenceClassification.from_pretrained(
        "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )

    model6.summary()

    tf_train_set6 = model6.prepare_tf_dataset(
        tokenized_train_ds,
        shuffle=True,
        batch_size=16,
        collate_fn=data_collator,
    )

    tf_validation_set6 = model6.prepare_tf_dataset(
        tekenized_validation_ds,
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator,
    )
    model6.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m]) # No loss argument!
    # metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set6)
    push_to_hub_callback = PushToHubCallback(
        output_dir="IC_sexto",
        tokenizer=tokenizer,
    )
    checkpoint = ModelCheckpoint(filepath = 'model6.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                                monitor = 'val_accuracy',
                                verbose = 1,
                                save_best_only = True,
                                mode = 'max')
    callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
with tf.device('/CPU:0'):
    model6.fit(x=tf_train_set6, validation_data=tf_validation_set6, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model6.history.history['val_accuracy']
val_f1_m = model6.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 7

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
with tf.device('/CPU:0'):
    model7 = TFAutoModelForSequenceClassification.from_pretrained(
        "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )

    tf_train_set7 = model7.prepare_tf_dataset(
        tokenized_train_ds,
        shuffle=True,
        batch_size=16,
        collate_fn=data_collator,
    )

    tf_validation_set7 = model7.prepare_tf_dataset(
        tekenized_validation_ds,
        shuffle=False,
        batch_size=16,
        collate_fn=data_collator,
    )
    model7.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
    # metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set7)
    push_to_hub_callback = PushToHubCallback(
        output_dir="IC_setimo",
        tokenizer=tokenizer,
    )
    checkpoint = ModelCheckpoint(filepath = 'model7.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                                monitor = 'val_accuracy',
                                verbose = 1,
                                save_best_only = True,
                                mode = 'max')
    callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
with tf.device('/CPU:0'):
    model7.fit(x=tf_train_set7, validation_data=tf_validation_set7, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model7.history.history['val_accuracy']
val_f1_m = model7.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 8

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model8 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set8 = model8.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set8 = model8.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model8.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set8)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_oitavo",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model8.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model8.fit(x=tf_train_set8, validation_data=tf_validation_set8,--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model8.history.history['val_accuracy']
val_f1_m = model8.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 9

In [ ]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model9 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set9 = model9.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set9 = model9.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model9.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set9)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_nono",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model9.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model9.fit(x=tf_train_set9, validation_data=tf_validation_set9, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model9.history.history['val_accuracy']
val_f1_m = model9.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 10

In [ ]:
batch_size = 16
num_epochs = 7
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=1e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model10 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set10 = model10.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set10 = model10.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model10.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set10)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_10",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model10.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model10.fit(x=tf_train_set10, validation_data=tf_validation_set10, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model10.history.history['val_accuracy']
val_f1_m = model10.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 11

In [ ]:
batch_size = 16
num_epochs = 7
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model11 = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-large-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set11 = model11.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set11 = model11.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model11.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set11)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_11",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model11.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model11.fit(x=tf_train_set11, validation_data=tf_validation_set11, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model11.history.history['val_accuracy']
val_f1_m = model11.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 12

In [ ]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=5e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model12 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set12 = model12.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set12 = model12.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model12.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set12)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_12",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model12.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model12.fit(x=tf_train_set12, validation_data=tf_validation_set12, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model12.history.history['val_accuracy']
val_f1_m = model12.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 13

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model13 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set13 = model13.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set13 = model13.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model13.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set13)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_13",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model13.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model13.fit(x=tf_train_set13, validation_data=tf_validation_set13, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model13.history.history['val_accuracy']
val_f1_m = model13.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)

# MODEL 14

In [ ]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=6e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model14 = TFAutoModelForSequenceClassification.from_pretrained(
    "roberta-large", num_labels=2, id2label=id2label, label2id=label2id
)

tf_train_set14 = model14.prepare_tf_dataset(
    tokenized_train_ds,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set14 = model14.prepare_tf_dataset(
    tekenized_validation_ds,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
model14.compile(optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])  # No loss argument!
# metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set14)
push_to_hub_callback = PushToHubCallback(
    output_dir="IC_14",
    tokenizer=tokenizer,
)
checkpoint = ModelCheckpoint(filepath = 'model14.epoch{epoch:02d}-accuracy{val_accuracy:.5f}.tf',
                             monitor = 'val_accuracy',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max')
callbacks = [checkpoint, push_to_hub_callback]

In [ ]:
model14.fit(x=tf_train_set14, validation_data=tf_validation_set14, epochs=num_epochs, callbacks=callbacks)

In [ ]:
val_acc = model14.history.history['val_accuracy']
val_f1_m = model14.history.history['val_f1_m']
print(val_acc)
print(val_f1_m)